<a href="https://colab.research.google.com/github/PheelaV/mlp-g066-finqa-mamba/blob/benchmarks/benchmarks/benchmarks_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install peft
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.8 MB/s eta 0:00:00


In [2]:
#clone the FinNLP repository
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

import sys
sys.path.append('/content/FinNLP/')

Cloning into 'FinNLP'...
remote: Enumerating objects: 1403, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 1403 (delta 240), reused 431 (delta 223), pack-reused 941
Receiving objects: 100% (1403/1403), 4.97 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (613/613), done.


In [3]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

from peft import PeftModel
import torch

# Load benchmark datasets from FinNLP
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-14m", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m")

# inputs = tokenizer("Hello, I am", return_tensors="pt")
# tokens = model.generate(**inputs)
# tokenizer.decode(tokens[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
batch_size = 32
# Check if the tokenizer has a pad token, if not, set it to the EOS token
if tokenizer.pad_token is None:
    print("Setting pad_token to eos_token.")
    tokenizer.pad_token = tokenizer.eos_token
device = torch.device("cuda")
model.cuda()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 128)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=128, out_features=384, bias=True)
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=128, out_features=512, bias=True)
          (dense_4h_to_h): Linear(in_features=512, out_features=128, bias=True)
          (act)

In [13]:
# FPB, len 1212
res = test_fpb(model, tokenizer, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 32. Total steps: 38


100%|██████████| 38/38 [01:49<00:00,  2.89s/it]

Acc: 0.6014851485148515. F1 macro: 0.25064466219700876. F1 micro: 0.6014851485148515. F1 weighted (BloombergGPT): 0.4522771255980678. 


In [14]:
res = test_tfns(model, tokenizer, batch_size = batch_size)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 32. Total steps: 75


100%|██████████| 75/75 [03:58<00:00,  3.18s/it]

Acc: 0.6553601340033501. F1 macro: 0.26393456446580654. F1 micro: 0.6553601340033501. F1 weighted (BloombergGPT): 0.5192481506952928. 


In [15]:
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 32. Total steps: 9


100%|██████████| 9/9 [00:28<00:00,  3.17s/it]

Acc: 0.07636363636363637. F1 macro: 0.04729729729729729. F1 micro: 0.07636363636363637. F1 weighted (BloombergGPT): 0.010835380835380836. 


In [16]:
res = test_nwgi(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/16184 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4047 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 32. Total steps: 127


 31%|███▏      | 40/127 [01:50<03:59,  2.75s/it]


KeyboardInterrupt: 